In [1]:
from __future__ import print_function

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import mshr
import numpy as np
import scipy.integrate as integrate
from fenics import *
import pickle

# local imports
from modules.mesh_utils import get_mesh
from modules.physics_utils import get_VandE, compute_trajectory, coord_impact, intersection
#from modules.plotting_utils import ?

# Parametres

In [ ]:
mesh_dict = { # we need L < L_mot
    'L_mot' : .005,
    'l_mot' : .003,
    'L_1' : .003, # dim_trou = L_mot-L = .002 ici
    'l_1': .001,
    'L_2' : .004, # dim_trou = .001 ici
    'l_2' : .002,
    'delta_vert_12' : .001,
    'L_vacuum' : .05,
    'l_vacuum': .02,
    'mesh_resolution' : 100,
    'refine_mesh' : True,
}

phi_dict = {
    'Phi_top_mot' : 0,
    'Phi_bord_mot': 'N',
    'Phi_electrode1' :30,
    'Phi_inter_electrode':'N',
    'Phi_electrode2':300,
    'Phi_sup_vacuum':'N',
    'Phi_inf_vacuum':'N',
}

physics_consts_dict = {
    'rhoelec': 0,
    'l_rho':0, #dist between rho and 0 // <l_2 // if 0, we consider rhoelec uniform
    'PERMITTIVITY' : 8.54e-12,
    'CHARGE':1.6e-19, 
    'M_NUCLEON':1.7e-27,
}

integration_parameters_dict = { #Best ==> 'tmax' : .00002 and 'dt' : .00000001 and '%Nout':100
    'tmax' : .000007,
    'dt' : .00000005,
    '%Nout': 95,
}

injection_dict = {
    'Nombre de particules':300,
    'proportion de I':0,
    'proportion de I+':0,
    'proportion de I-':1,
    'débit de particule en entrée de la grille':1e12,
    'gamma':1,
    'Vion':2000,
    'Vneutre':200,
    'Sigma_Vion':500,
    'Sigma_Vneutre':80,
}

mode_dict={
    'Choc symétrique?':False,
    'Contact inter particules?':False,
    'coef inelasticite':0.1, 
    'coef scattering':0.3,
    'proba perte q par choc':1, 
    'X0 fixe ?':False,
}

In [7]:
liste_L_2=[.0025, .00275, .003, .00325, .0035, .00375, .004, .00425, .0045, .0048] #liste_L_2=[.0025, .003, .0035, .004 , .0045, .0048] #6 valeurs 
liste_l_2=[.001, .0025, .004, .006, .008, .01, .02, .03, .04,   .05] #liste_l_2=[.001, .0025, .004, .006, .008, .01, 0.03, .05] #8 valeurs 
liste_couleur=plt.cm.plasma(np.linspace(0,1,len(liste_l_2)))

NameError: name 'inferno' is not defined

# Maj des listes 

In [3]:
with open("OPTIM_liste_param_testes.txt", "rb") as fp:
    liste_param_testes = pickle.load(fp)
with open("OPTIM_liste_Nb_out.txt", "rb") as fp:
    liste_Nb_out = pickle.load(fp)
with open("OPTIM_liste_pf.txt", "rb") as fp:
    liste_pf = pickle.load(fp)
with open("OPTIM_liste_alpha_moy.txt", "rb") as fp:
    liste_alpha_moy = pickle.load(fp)
with open("OPTIM_liste_alpha_sigma.txt", "rb") as fp:
    liste_alpha_sigma = pickle.load(fp)
with open("OPTIM_liste_V_moy.txt", "rb") as fp:
    liste_V_moy = pickle.load(fp)
with open("OPTIM_liste_V_sigma.txt", "rb") as fp:
    liste_V_sigma = pickle.load(fp)

In [13]:
'''liste_param_testes = []
liste_Nb_out = []
liste_pf = []
liste_alpha_moy = []
liste_alpha_sigma = []
liste_V_moy = []
liste_V_sigma = []'''

In [7]:
for l_2 in liste_l_2:
    for L_2 in liste_L_2:

        param=[L_2, l_2]
            
        if param in liste_param_testes:
            print('combinaison '+str(param)+' déjà testée')

        else:    
            mesh_dict['L_2'] = L_2
            mesh_dict['l_2'] = l_2
            
            if L_2 >= .004 or l_2 >= .008:
                integration_parameters_dict['tmax']=.00001
            
            mesh, segments_list, zone = get_mesh(mesh_dict)
            Phi, E, f = get_VandE(mesh, mesh_dict, phi_dict, physics_consts_dict)

            Nb_out, pf_3_especes, alpha_moy_3_especes, alpha_sigma_3_especes, V_moy_3_especes, V_sigma_3_especes = \
            compute_trajectory(integration_parameters_dict, injection_dict, mesh_dict, mode_dict, segments_list, zone,E, physics_consts_dict, False)

            liste_Nb_out.append(Nb_out)
            liste_pf.append(pf_3_especes)
            liste_alpha_moy.append(alpha_moy_3_especes)
            liste_alpha_sigma.append(alpha_sigma_3_especes)
            liste_V_moy.append(V_moy_3_especes)
            liste_V_sigma.append(V_sigma_3_especes)
            liste_param_testes.append(param)

            print('combinaison '+str(param)+' testée')
            print('------------------------------------------------------')
            
            integration_parameters_dict['tmax']=.000007

            with open("OPTIM_liste_param_testes.txt", "wb") as fp:
                pickle.dump(liste_param_testes, fp)
            with open("OPTIM_liste_Nb_out.txt", "wb") as fp:
                pickle.dump(liste_Nb_out, fp)
            with open("OPTIM_liste_pf.txt", "wb") as fp:
                pickle.dump(liste_pf, fp)
            with open("OPTIM_liste_alpha_moy.txt", "wb") as fp:
                pickle.dump(liste_alpha_moy, fp)
            with open("OPTIM_liste_alpha_sigma.txt", "wb") as fp:
                pickle.dump(liste_alpha_sigma, fp)
            with open("OPTIM_liste_V_moy.txt", "wb") as fp:
                pickle.dump(liste_V_moy, fp)
            with open("OPTIM_liste_V_sigma.txt", "wb") as fp:
                pickle.dump(liste_V_sigma, fp)

combinaison [1, 0.005, 0.037, 0.0045, 0.003, 30, 300] déjà testée
début d'initialisation
début de l'injection
toutes les particules sont injectées


KeyboardInterrupt: 

In [ ]:
##On complete les listes en changeant les 'None' (si pas de particules du type en sortie) en 0.
#pour les vitesses on le verra pas, pour les angles ça n'influence pas

for i in range (len(liste_param_testes)):
    if liste_V_moy[i][0] == None:
        liste_V_moy[i][0] = 0
        liste_alpha_moy[i][0] = 0
        liste_V_sigma[i][0] = 0
        liste_alpha_sigma[i][0] = 0

In [ ]:
liste_param_testes = np.array(liste_param_testes)
liste_Nb_out = np.array(liste_Nb_out)
liste_pf = np.array(liste_pf)
liste_alpha_moy = np.array(liste_alpha_moy)
liste_alpha_sigma = np.array(liste_alpha_sigma)
liste_V_moy = np.array(liste_V_moy)
liste_V_sigma = np.array(liste_V_sigma)

# Plot

In [ ]:
liste_indice_rslt_l_2=[] #contient des listes d'indices globaux de liste_param_testes pour lesquels 
                        #l_2 est celui de la même place que ds liste_l_2
for l_2 in liste_l_2:
    liste_intermediaire=[]
    for i in range(len(liste_param_testes)):
        if liste_param_testes[i,1]==l_2:
            liste_intermediaire.append(i)
    liste_indice_rslt_l_2.append(liste_intermediaire)
    
liste_indice_rslt_l_2=np.array(liste_indice_rslt_l_2)

In [ ]:
liste_couple=[[.003,0.001],
              [0.004,0.001],
              [0.003,0.01],
              [0.004,0.01]]
liste_indice_cas_test=[None,None,None,None] #contient les indices globaux de liste_param_testes pour lesquels 
                                            #les dimensions sont celles des cas tests
for i in range(len(liste_param_testes)):
    param=liste_param_testes[i]
    for j in range(4):
        couple=liste_couple[j]
        if (couple==param).all():
            liste_indice_cas_test[j]=i

# N°1

In [ ]:
plt.figure(figsize=(10,10))

for i in range(len(liste_l_2)):
    liste_intermediaire = liste_indice_rslt_l_2[i]
    abscisse = liste_param_testes[liste_intermediaire, 0]
    ordonnee = liste_pf[liste_intermediaire,0]
    nom = 'l_2 = ' + str(liste_l_2[i]) + 'm'
    c = liste_couleur[i]
    plt.plot(abscisse, ordonnee, label=nom, color=c, markersize=5, linewidth=1, marker='o')

for indice in liste_indice_cas_test:
    abscisse = liste_param_testes[indice, 0]
    ordonnee = liste_pf[indice,0]
    plt.scatter(abscisse, ordonnee, marker='*', s=100, color='k')

plt.xlabel("L_2 (m)",size=30)
plt.ylabel("taux de neutralisation en sortie",size=30)
plt.ylim(-0.05,1)
plt.legend()
plt.text(.0025,-0.3,'300 particules, scattering=0.3, proba_perte_charge=1, eta=0.1, \nchocs non symétriques, sans collision, gamma=1, rho=0', size=15)

plt.show()

# N°2

In [ ]:
#rappel, liste_l_2=[.001, .0025, .004, .006, .008, .01, .02, .03, .04, .05] #10 valeurs
indice_l_2_choisi = 5
liste_indice_rslt_l_2_choisi = liste_indice_rslt_l_2[indice_l_2_choisi]

abscisse=liste_param_testes[liste_indice_rslt_l_2_choisi, 0]
ordonnee=liste_V_moy[liste_indice_rslt_l_2_choisi,0]*liste_pf[liste_indice_rslt_l_2_choisi,0] + liste_V_moy[liste_indice_rslt_l_2_choisi,2]*liste_pf[liste_indice_rslt_l_2_choisi,2]
barre=liste_V_sigma[liste_indice_rslt_l_2_choisi,0]*liste_pf[liste_indice_rslt_l_2_choisi,0] + liste_V_sigma[liste_indice_rslt_l_2_choisi,2]*liste_pf[liste_indice_rslt_l_2_choisi,2]
barre=1.96*barre*np.power(liste_Nb_out[liste_indice_rslt_l_2_choisi],-.5)
c=liste_couleur[indice_l_2_choisi]

plt.figure(figsize=(10,10))
plt.scatter(abscisse, ordonnee, color=c)
plt.errorbar(abscisse, ordonnee, yerr=barre, fmt = 'none', capsize = 7, elinewidth=1, ecolor='k')
plt.xlabel('L_2 (m)',size=30)
plt.ylabel('V (m/s) total en sortie',size=30)   
plt.ylim(14000,20000)
for i in range (1,4):
    plt.axhline(y=20000*0.9**i, linestyle='--',color='grey')    
plt.text(0.0042,19500,'l_2 fixé à '+str(liste_l_2[indice_l_2_choisi])+'m', size=15)
plt.text(0.0025,13000,'300 particules, scattering=0.3, proba_perte_charge=1, eta=0.1, \nchocs non symétriques, sans collision, gamma=1, rho=0', size=15)

plt.show()

# N°3

In [ ]:
#rappel, liste_l_2=[.001, .0025, .004, .006, .008, .01, .02, .03, .04, .05] #10 valeurs
indice_l_2_choisi = 8
liste_indice_rslt_l_2_choisi = liste_indice_rslt_l_2[indice_l_2_choisi]

abscisse=liste_param_testes[liste_indice_rslt_l_2_choisi, 0]
ordonnee=liste_alpha_moy[liste_indice_rslt_l_2_choisi,0]*liste_pf[liste_indice_rslt_l_2_choisi,0] + liste_alpha_moy[liste_indice_rslt_l_2_choisi,2]*liste_pf[liste_indice_rslt_l_2_choisi,2]
barre=liste_alpha_sigma[liste_indice_rslt_l_2_choisi,0]*liste_pf[liste_indice_rslt_l_2_choisi,0] + liste_alpha_sigma[liste_indice_rslt_l_2_choisi,2]*liste_pf[liste_indice_rslt_l_2_choisi,2]
barre=1.96*barre*np.power(liste_Nb_out[liste_indice_rslt_l_2_choisi],-.5)
c=liste_couleur[indice_l_2_choisi]

fig = plt.figure(figsize=(10,10))
ax  = fig.add_subplot(111)
plt.scatter(abscisse, ordonnee, color=c)
plt.errorbar(abscisse, ordonnee, yerr=barre, fmt = 'none', capsize = 7, elinewidth=1, ecolor='k')
y_pi   = ordonnee/np.pi
unit   = 0.05
y_tick = np.arange(-0.05, 0.051, unit)
y_label = [r"$-\frac{\pi}{20}$", r"$0$", r"$+\frac{\pi}{20}$"]
ax.set_yticks(y_tick*np.pi)
ax.set_yticklabels(y_label, fontsize=20)
plt.xlabel('L_2 (m)',size=30)
plt.ylabel('angle (rad) du flux total en sortie',size=30)   
plt.ylim(-1.1*np.pi/20,1.1*np.pi/20)
plt.axhline(y=0, linestyle='--',color='grey')  
plt.text(0.0042,0.15,'l_2 fixé à '+str(liste_l_2[indice_l_2_choisi])+'m', size=15)
plt.text(0.0025,-0.24,'300 particules, scattering=0.3, proba_perte_charge=1, eta=0.1, \nchocs non symétriques, sans collision, gamma=1, rho=0', size=15)

plt.show()

# N°4

In [6]:
X=np.array(liste_L_2)
Y=np.array(liste_l_2)
X,Y=np.meshgrid(X,Y)
tab_pf=np.zeros(np.shape(X))
tab_V_moy=np.zeros(np.shape(X))
tab_alpha_sigma=np.zeros(np.shape(X))

for i in range(len(tab_V_moy)):
    for j in range(len(tab_V_moy[0])):
        ind=None
        for k in range(len(liste_param_testes)):
            if (liste_param_testes[k] == (X[i,j],Y[i,j])).all():
                ind = k
                break
        tab_pf[i,j]=liste_pf[ind,0]
        tab_V_moy[i,j]=liste_V_moy[ind,0]*liste_pf[ind,0]+liste_V_moy[ind,2]*liste_pf[ind,2]
        tab_alpha_sigma[i,j]=(liste_alpha_sigma[ind,0]*liste_pf[ind,0]+liste_alpha_sigma[ind,2]*liste_pf[ind,2])*1.96*liste_Nb_out[ind]**(-.5)
        
plt.figure(figsize=(20,7))

plt.subplot(1,3,1)
plt.pcolormesh(X,Y,tab_pf,cmap='plasma')
plt.xlabel('L_2 (m)',size=25)
plt.ylabel('l_2 (m)',size=25)
plt.title('proportion de neutre',size=25)
plt.colorbar()

plt.subplot(1,3,2)
plt.pcolormesh(X,Y,tab_V_moy,cmap='plasma')
plt.xlabel('L_2 (m)',size=25)
plt.ylabel('l_2 (m)',size=25)
plt.title('vitesse moyenne (m/s)',size=25)
plt.colorbar()

plt.subplot(1,3,3)
plt.pcolormesh(X,Y,tab_alpha_sigma,cmap='plasma')
plt.xlabel('L_2 (m)',size=25)
plt.ylabel('l_2 (m)',size=25)
plt.title("1.96*sigma/sqrt(N) de \nl'angle (rad) du flux",size=25)
plt.colorbar()

plt.text(-.0034,-0.01,'300 particules, scattering=0.3, proba_perte_charge=1, eta=0.1, chocs non symétriques, sans collision, gamma=1, rho=0', size=15)
plt.show()

TypeError: 'int' object is not subscriptable